# Study of the Swiss electrical mix (and imports) between 2016 and 2024

This notebook is a study of the Swiss electrical mix (production and imports) between 2016 and 2024. It aims to represent the repartition of the consumed electricity and the associated ghg emissions.
It illustrates some possible uses of EcoDynElec.

We perform a historical analysis of the Swiss energy mix over 8 years, between 2016 and 2024. We have hourly European electricity production and exchange data, from which we calculate the Swiss electricity mix and its carbon impact (in grams of CO2 per kWh).
We mainly use two information types:
- The **producer mix** refers to the electricity produced in the designated country. It contains only local sources. When converting the producer mix into kWh, we obtain the **electricity production** of the country.
- The **consumer mix** refers to the electricity consumed in the designated country. It contains local sources as well as the import of electricity from other countries. When converting the consumer mix into kWh, we obtain the **electricity consumption** of the country.

These data make it possible to calculate the **carbon impact hour by hour** of the electricity produced and consumed in the country (these two impacts are therefore different), and then to study the sources of these emissions.

## 0. Configuring EcoDynElec and loading the data

In [ ]:
# Autoreload imports
%load_ext autoreload
%autoreload 2

# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import sys
sys.path.insert(0, r"C:\Users\...\Desktop")

from ecodynelec.parameter import Parameter  # Import the class to manipulate parameters
from ecodynelec.preprocessing.downloading import download #import the class to download entsoe data
from examples.mix_analysis.analysis_functions import format_data_0, get_metrics, plot_hourly_heatmap, plot_years, \
    plot_typical_days

entsoe_id = pd.read_excel('../support_files/entsoe_id.xlsx', index_col=0, header=None)

# change plt font size
plt.rcParams.update({'font.size': 12})

In [ ]:
# EcoDynElec configuration
my_config = Parameter()
my_config.freq = "H"
my_config.ctry = ['AT', 'CH', 'DE', 'FR', 'IT']
my_config.target = ['CH']  # You can put multiple targets here
# Enable this if you want to plot fig 6 (will impact the computation time - it took me 15min)
my_config.target = ['CH', 'AT', 'FR', 'DE','IT']
my_config.data_cleaning = True
my_config.sg_imports = True

# Dynamic impact for STEP
my_config.dynamic_impact = True
my_config.interpolated_stock = True # if False dynamic impact = Mix CH, if True storage data are interpolated from week to hour

# Choose Between EcoInvent and UVEK for the LCA database
my_config.uvek_data = False # if True LCA database = UVEK else LCA database = EcoInvent

# Indicate where to save generation data
my_config.path.generation = "./test_data/downloads/generations/"
# Indicate where to save exchange data
my_config.path.exchanges = "./test_data/downloads/exchanges/"
# Recommended config: include swiss grid residual (difference between swiss grid and entso-e data)
my_config.residual_global = True
# Recommended config: include real (2020-2022) and estimated (2016-2019) renewable production (using EcoDynElec-Enr-Model)
my_config.ch_enr_model_path = '../support_files/enr_prod_2016-2024_completed.csv'
# Recommended config : include storage data to obtain dynamic impact
my_config.storage_path = "../ecodynelec/data/"
# Recommended config : include UI network data
my_config.path.ui_network = '../support_files/Unit_Impact_Network.xlsx'

# Years to study
years = ['2016','2017','2018','2019','2020','2021','2022','2023','2024']

In [ ]:
# Data downloading configuration
### Set the dates (to select files to download)
my_config.start = '2023-01-01 00:00:00'
my_config.end = '2024-12-31 23:45:00'

### Configure the server connection
my_config.server.useServer = False # Turn it to True if you want to download data from the server

### Credentials : Make sure to enter your information in the entsoe_id.xlsx file in the support_files folder.
my_config.server.username = entsoe_id.loc['Mail'].iloc[0]
my_config.server.password = entsoe_id.loc['Password'].iloc[0]

# Download the data 
#download(my_config, threshold_minutes=15, threshold_size=0.9, is_verbose=True)

In [ ]:
from examples.mix_analysis.analysis_functions import generate_data

## Allows to run this notebook from another notebook without generating the data again (using %run)
if 'skip_generation' not in globals():
    flows, prods, mixs, prod_impacts, impacts = generate_data(my_config, years,
                                                              savedir="./mix_analysis/results_local_all_enr/")
else:
    print("Skipping data generation")

If examples/test_data/downloads and examples/mix_analysis/results_local_all_enr are not empty, there is no need to execute the cell below.

#### 0.2 Load data from saved files
If the files for the years of interest have already been generated, they can be loaded directly in this section

In [ ]:
from examples.mix_analysis.analysis_functions import load_data

flows, prods, mixs, prod_impacts, impacts = load_data(my_config.target, my_config.freq, years,
                                                      savedir="./mix_analysis/results_local_all_enr/")

### 0.3 Pre-processing the data
Here we reformat the data to make it easier to manipulate and plot

In [ ]:
from examples.mix_analysis.analysis_functions import concatenate_and_format_data

# NOTE: Unless you use multiple targets (see section 0.), these dicts will contain only one element, corresponding to the target country
raw_productions_by_src, raw_consumptions_by_src, electricity_prod_mixs, electricity_mixs, producing_electricity_impacts, electricity_impacts = concatenate_and_format_data(my_config.target, years, flows, prods, mixs, prod_impacts, impacts)

# Extract the data for CH
raw_production_by_src = raw_productions_by_src['CH']
raw_consumption_by_src = raw_consumptions_by_src['CH']
electricity_prod_mix = electricity_prod_mixs['CH']
electricity_mix = electricity_mixs['CH']
electricity_impact = electricity_impacts['CH']
producing_electricity_impact = producing_electricity_impacts['CH']

### 0.4 Plots configuration

Some color codes for each country

In [ ]:
ordered_countries = ['CH', 'DE', 'FR', 'AT', 'IT', 'Other']
colormap = {
    'sum': 'gray',
    'CH': '#1f77b4',
    'DE': '#c62728',
    'FR': '#2ca02c',
    'IT': '#9467bd',
    'AT': '#ff7f0e',
    'Other': '#e377c2'
}
colormap_extended = colormap | {
    'Nuclear_CH': '#8c564b',
    'Residual_Hydro_Run-of-river_and_poundage_CH': '#000080',  # Couleur bleu marine
    'Residual_Hydro_Water_Reservoir_CH': '#004090',  # Couleur bleu marine
    'Hydro_Water_Reservoir_CH': '#98df8a',
    'Residual_Other_CH': '#ffbb78',
    'Hydro_Pumped_Storage_CH': '#bcbd22',
    'Hydro_Run-of-river_and_poundage_CH': '#17becf',
    'Solar_CH': '#d62728',
    'Other_CH': '#1f77b4'
}
ordered_colors = [colormap[c] for c in ordered_countries]

## 1. Inter-annual analysis

Swiss annual electricity consumption

In [ ]:
yearly_cons = raw_consumption_by_src['raw_df'].resample('Y').sum().sum(axis=1).map(lambda x: f'{x / 1000:.2f} GWh')
yearly_cons.index = pd.Series(years)
print('Consumption:')
print(yearly_cons)

yearly_prod = raw_production_by_src['raw_df'].resample('Y').sum().sum(axis=1).map(lambda x: f'{x / 1000:.2f} GWh')
yearly_prod.index = pd.Series(years)
print('Production:')
print(yearly_prod)

Monthly electricity consumption in 2021

In [ ]:
monthly_cons = raw_consumption_by_src['raw_df']
monthly_cons = monthly_cons.loc[[d.year == 2021 for d in monthly_cons.index]]
monthly_cons = monthly_cons.resample('ME').sum().sum(axis=1).map(
    lambda x: x / 1000)  #.map(lambda x: f'{x / 1000:.2f} GWh')
monthly_cons.index = [f'{d.month_name()}' for d in monthly_cons.index]
#print(monthly_prod)
monthly_cons.plot(figsize=(12, 4), title='Monthly electricity consumption in Switzerland in 2021')
plt.xlabel('Time interval: month')
plt.ylabel('Electricity consumption (GWh)');

2021 Monthly production from selected sources (helps to validate the data)

In [ ]:
from ecodynelec.pipeline_functions import get_producing_mix_kwh, get_consuming_mix_kwh

ch_prod_raw = {}
# Multi-target: correctly extract the data for each target country
if len(my_config.target) > 1:
    for y in years:
        ch_prod_raw[y] = get_producing_mix_kwh(flows_df=flows[y]['CH'], prod_mix_df=prods[y]['CH'])
else:
    for y in years:
        ch_prod_raw[y] = get_producing_mix_kwh(flows_df=flows[y], prod_mix_df=prods[y])
ch_prod_raw = format_data_0({y: ch_prod_raw[y] for y in years})['raw_df']
cols = ['Solar_CH', 'Nuclear_CH',
        'Wind_Onshore_CH']  # ['Residual_Hydro_Water_Reservoir_CH', 'Residual_Hydro_Run-of-river_and_poundage_CH', 'Residual_Other_CH'] ...
start = '2021-01-01'
end = '2021-12-31'
monthly_prod = ch_prod_raw[cols][start:end].resample('ME').sum()
del ch_prod_raw
monthly_prod / 1000

Main power sources in the Swiss electricity mix

In [ ]:
yearly_mix = electricity_mix['raw_df'].resample('YE').mean()
local_cols = yearly_mix.mean().nlargest(20).index.tolist()
yearly_mix = yearly_mix.applymap(lambda x: f'{x * 100:.0f} %')
yearly_mix.index = pd.Series(years)
print(yearly_mix[local_cols])

Consumed electricity by country of origin and associated GHG emissions, monthly view

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(18, 5))
ax1.set_title("Monthly profile of the Swiss electricity consumer mix from 2016 to 2024")
mix_import = electricity_mix['df'].drop(['sum'], axis=1)
mix_import = mix_import.multiply(raw_consumption_by_src['df']['sum'], axis='index').resample('M').sum() / 1000
mix_import[ordered_countries].plot.area(ax=ax1, color=ordered_colors, legend=False)
ax1.set_xlabel('Time interval: month')
ax1.set_ylabel('Monthly total Swiss consumption (GWh)')
#plot_years()
fig.legend()
fig.legend()
plt.grid(True)
plt.show()

fig, ax1 = plt.subplots(1, 1, figsize=(18, 4))
ax1.set_title("Monthly profile of the Swiss electricity consumer mix climate change impact, 2016 to 2024")
mix_import = electricity_impact['df'].drop(['sum'], axis=1)
mix_import = mix_import.resample('ME').mean()
mix_import[ordered_countries].plot.area(ax=ax1, color=ordered_colors, legend=False)
ax1.set_xlabel('Time interval: month')
ax1.set_ylabel('Monthly average of GHG emissions (gCO2eq/kWh)')
#plot_years()
fig.legend()
plt.grid(True)
plt.show()

Consumed electricity by country of origin and associated GHG emissions, weekly view

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(18, 4))
ax1.set_title("Origins of weekly Swiss consumer mix and associated GHG emissions")
mix_import = electricity_mix['df'].drop(['sum'], axis=1)
mix_import = mix_import.multiply(raw_consumption_by_src['df']['sum'], axis='index').resample('W').sum() / 1000
mix_import[ordered_countries].plot.area(ax=ax1, color=ordered_colors, legend=False)
ax1.set_xlabel('Time interval: week')
ax1.set_ylabel('Weekly total Swiss consumption (GWh)')

ax2 = ax1.twinx()
ch_consumer_impact = (electricity_impact['df']['sum']).resample('W').mean()  # convert kgCO2eq/kWh -> gCO2eq/kWh
ch_consumer_impact.plot(ax=ax2, linestyle='-', color=[colormap['sum']])
de_producing_impact = (producing_electricity_impacts['DE']['df']['sum']).resample(
    'W').mean()  # convert kgCO2eq/kWh -> gCO2eq/kWh
de_producing_impact.plot(ax=ax2, linestyle='--', color=[colormap['sum']])
ax2.set_ylabel('Weekly average of GHG emissions (gCO2eq/kWh)')

plot_years()

fig.legend(np.concatenate([ax1.get_lines(), ax2.get_lines()]),
           np.concatenate([ax1.get_legend_handles_labels()[1],
                           ['CH consumer mix (GHG emissions)', 'DE production mix (GHG emissions)']]),
           loc='upper left', ncol=4, bbox_to_anchor=(0.1, 0));

In [ ]:
mix_import = mix_import.multiply(raw_consumption_by_src['df']['sum'], axis='index').resample('YE').sum() / 1000
pd.DataFrame(mix_import)

Impact of the Swiss consumer mix per country producing the electricity

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(18, 4))
ax1.set_title("Weekly GHG emissions per kWh of the Swiss consumer mix - per country of origin 2016-2024")
impc = electricity_impact['df'].drop(['sum'], axis=1)
impc = impc.resample('W').mean()
impc[ordered_countries].plot.area(ax=ax1, color=ordered_colors, legend=False)
ax1.set_xlabel('Time interval: week')
ax1.set_ylabel('GHG emissions (gCO2eq/kWh)')
plot_years()
fig.legend();

Impact of the Swiss consumer mix aggregated per electricity production type

In [ ]:
def compute_per_type(results):
    """Function to group datasets per type of unit, regardless of the country of origin"""
    unit_list = np.unique(
        [k[:-3] if k[-3] == "_" else k for k in results.columns])  # List the different production units

    per_unit = []
    for u in unit_list:
        cols = [k for k in results.columns if k[:-3] == u]  # collect the useful columns
        per_unit.append(pd.Series(results.loc[:, cols].sum(axis=1), name=u))  # aggregate

    return pd.concat(per_unit, axis=1)
electricity_impact['raw_df'][electricity_impact['raw_df'] < 0] = 0
imp = electricity_impact['raw_df'].loc['2021'].resample('D').mean()
es13_per_type = compute_per_type(imp)  # Group Climate Change index impacts per country
local_cols = es13_per_type.mean().nlargest(9).index.tolist()
es13_per_type_10 = es13_per_type[local_cols]
es13_per_type_10['Other sources'] = es13_per_type.sum(axis=1) - es13_per_type_10.sum(axis=1)
es13_per_type_10.plot.area(figsize=(15, 5),
                           title="Daily GHG emissions of the Swiss consumer mix - per production type in 2021")  # Build the graph

plt.xlabel('Time interval: day')
plt.ylabel('GHG emissions (gCO2eq/kWh)');

Mix by source (import source and main CH production sources)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

# import mix
compute_mix = electricity_mix['df'][ordered_countries].drop('CH', axis=1)
# local CH mix
local_mix = electricity_mix['raw_df']  # wtf .resample('W').mean()
local_mix = local_mix.drop(local_mix.loc[:, [not k.endswith('CH') or 'Mix' in k for k in local_mix.columns]], axis=1)
local_cols = local_mix.mean().nlargest(len(ordered_countries)).index.tolist()
print('lc cols', local_cols)
local_mix['Other_CH'] = local_mix.drop(local_cols, axis=1).sum(axis=1)
# merged mix
compute_mix = pd.concat([compute_mix, local_mix[local_cols], local_mix['Other_CH']], axis=1)
metric = 'mean'
metric_values = get_metrics(years, compute_mix, [metric], 'Y')
colors = [colormap_extended[c] for c in compute_mix.columns]
# yearly consumption (TWh), used for comparison
yearly_consumption = raw_consumption_by_src['df']['sum'].resample('Y').sum() / 1e6
yearly_consumption.index = metric_values[metric].index  # keep same x axis
# plot
ax2 = ax.twinx()
ax = metric_values[metric].plot.bar(ax=ax, stacked=True, color=colors, legend=True)
yearly_consumption.plot(ax=ax2, linestyle='-.', color='gray', legend=False, label='Yearly Swiss consumption')
ax.set_title(f'Daily average electrical mix in Switzerland by source')
ax.set_label('date')
ax.set_ylabel('Share in total mix (%)')
ax.set_ylim(0, 1)
ax2.set_ylabel('Yearly Swiss consumption (TWh)')
ax2.set_ylim(40, 70)
# Merge legend of both axes
handles, labels = ax.get_legend_handles_labels()
handles.append(ax2.get_lines()[0])
labels.append(ax2.get_legend_handles_labels()[1][0])
ax.legend(handles, labels, ncol=3,
          loc='upper left', bbox_to_anchor=(0, -0.2));

Residual mix decomposition (difference between Entso-e breakdown and SwissGrid national production)

In [ ]:
# local CH mix
local_cols = ['Residual_Hydro_Run-of-river_and_poundage_CH', 'Residual_Hydro_Water_Reservoir_CH', 'Solar_CH', 'Wind_Onshore_CH', 'Residual_Other_CH', 'Biomass_CH', 'Waste_CH']
compute_mix = electricity_mix['raw_df'][local_cols]
metric = 'mean'
metric_values = get_metrics(years, compute_mix, [metric], 'Y')

colormap_extended['Wind_Onshore_CH'] = colormap['Other']#'#17becf'
colormap_extended['Biomass_CH'] = colormap['FR']
colormap_extended['Waste_CH'] = colormap_extended['Nuclear_CH']
colors = [colormap_extended[c] for c in compute_mix.columns]

# plot
fig, ax = plt.subplots(figsize=(12, 4))
ax = metric_values[metric].plot.bar(ax=ax, stacked=True, legend=True, color=colors)
ax.set_title(f'Daily average electrical mix in Switzerland by source')
ax.set_label('date')
ax.set_ylabel('Share in total mix (%)')
ax.set_ylim(0, 0.3)
# Merge legend of both axes
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, ncol=3,
          loc='upper left', bbox_to_anchor=(0, -0.2));

Impact of consumption mix over five years

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
data = electricity_impact['df'].astype('float32')
data = pd.DataFrame(data['sum'])
data['Year'] = [date.year for date in data.index]
grouped_data = data.groupby('Year')
seasonal_data = [grouped_data.get_group(season)[data.columns[0]] for season in grouped_data.groups]
ax.boxplot(seasonal_data)
ax.set_xticklabels(grouped_data.groups.keys())
ax.set_title('Hourly GHG emissions of Swiss consumption mix')
ax.set_xlabel('Year')
ax.set_ylabel('Hourly average of GHG emissions (gCO2eq/kWh)');

Impact of production mix over five years

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
data = producing_electricity_impact['raw_df'].astype('float32')
data = data.drop(data.loc[:, [not k.endswith('CH') or 'Mix' in k for k in data.columns]], axis=1)
data = pd.DataFrame(data.sum(axis=1))
data['Year'] = [date.year for date in data.index]
grouped_data = data.groupby('Year')
seasonal_data = [grouped_data.get_group(season)[data.columns[0]] for season in grouped_data.groups]
ax.boxplot(seasonal_data)
ax.set_xticklabels(grouped_data.groups.keys())
ax.set_title('Hourly GHG emissions of Swiss production mix')
ax.set_xlabel('Year')
ax.set_ylabel('Hourly average of GHG emissions (gCO2eq/kWh)');

Figure 6, comparison between the producing impacts of several countries and the import mix of Switzerland
**Make sure to compute the production mix of *all* the countries before running this cell (enable the corresponding line in section 0)**

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
x = np.linspace(0, 1, 100)
countries = ['FR', 'DE', 'AT', 'CH']
for c in countries:
    #mx = electricity_mixs['CH']['df'][c].resample('M').mean()
    mx = raw_consumptions_by_src['CH']['df'][c].resample('W').sum().map(lambda x: x / 1000)
    px = producing_electricity_impacts[c]['df']['sum'].resample('W').mean()
    plt.plot(mx, px, color=colormap[c], label=c, marker='x', linestyle='None')
plt.title('Avg carbon impact of production mix vs weekly imported electricity')
plt.xlabel('Imported electricity (GWh)')
plt.ylabel('GWP (gCO2eq/kWh)')
plt.legend();

Table 4, comparison between the producing impacts of several countries an the impacts of imports in Switzerland
**Make sure to compute the production mix of *all* the countries before running this cell (enable the corresponding line in section 0)**

In [ ]:
countries = ['FR', 'DE', 'AT']
table = {}
freq = 'Y'
for c in countries:
    table[c + '_prod'] = producing_electricity_impacts[c]['df']['sum'].resample(freq).mean()
for c in countries:
    emitted_co2 = producing_electricity_impacts[c]['df']['sum'].multiply(raw_consumptions_by_src['CH']['df'][c])
    emitted_co2 = emitted_co2.resample(freq).sum() / raw_consumptions_by_src['CH']['df'][c].resample(freq).sum()
    table[c + '_import'] = emitted_co2
    #table[c + '_import'] = electricity_impact['df'][c].resample(freq).mean()
tabled = pd.DataFrame(table)
for c in countries:
    tabled[c + '_diff'] = (tabled[c + '_import'].astype(float) - tabled[c + '_prod'].astype(float)) / tabled[
        c + '_prod'].astype(float) * 100
tabled = tabled.applymap(lambda val: f'{val:.2f}').T
ordered_table = tabled.loc[
    ['AT_prod', 'AT_import', 'AT_diff', 'FR_prod', 'FR_import', 'FR_diff', 'DE_prod', 'DE_import', 'DE_diff']]
ordered_table.index = ['AT production mix impact', 'Impact of AT electricity consumed in CH', 'Difference (%)',
                       'FR production mix impact', 'Impact of FR electricity consumed in CH', 'Difference (%)',
                       'DE production mix impact', 'Impact of DE electricity consumed in CH', 'Difference (%)']

In [ ]:
ordered_table

## 2. Annual statistics

Annual average of carbon emissions per produced kWh:

In [ ]:
yearly_mean = producing_electricity_impact['df'][['sum']].groupby(producing_electricity_impact['df'].index.year).mean()
yearly_mean

Annual average of carbon emissions per consumed kWh:

In [ ]:
yearly_mean = electricity_impact['df'][['sum']].groupby(electricity_impact['df'].index.year).mean()
yearly_mean

Quartiles and deciles of the given years

In [ ]:
ges = electricity_impact['df']['sum']
ges = pd.DataFrame(ges)
ges['Annee'] = [date.year for date in ges.index]
ges.iloc[-1, 0] = ges.iloc[-2]['sum']

In [ ]:
import pandas as pd

stats_dict = {'Année': [], 'Moyenne': [], 'Médiane': [], 'Quartile 1': [], 'Quartile 3': [], 'Décile 1': [], 'Décile 9': [], 'Min': [], 'Max': []}
for annee in ges['Annee'].unique():
    data_annee = ges[ges['Annee'] == annee]['sum']
    stats_dict['Année'].append(annee)
    stats_dict['Moyenne'].append(data_annee.mean())
    stats_dict['Médiane'].append(data_annee.median())
    stats_dict['Quartile 1'].append(data_annee.quantile(0.25))
    stats_dict['Quartile 3'].append(data_annee.quantile(0.75))
    stats_dict['Décile 1'].append(data_annee.quantile(0.1))
    stats_dict['Décile 9'].append(data_annee.quantile(0.9))
    stats_dict['Min'].append(data_annee.min())
    stats_dict['Max'].append(data_annee.max())

stats_df = pd.DataFrame(stats_dict)
#stats_df.to_csv('EDE_CH_stats_gwp_2016_2022.csv')
stats_df

## 3. Intra-annual analysis

Heatmaps of :
- CH consumption
- DE and FR imports
- Consumption from hydro production (pumped storage and water reservoir)

The exported electricity is not taken into account in these figures.

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(18, 12))
countries = ['sum', 'CH', 'DE', 'FR']
# Heatmap import
data = raw_consumption_by_src['df'][countries].copy()
val_max = data[countries].max().max()
for i in range(len(countries)):
    series = data[countries[i]]
    xlabels = list(series.resample('Y').mean().index.map(lambda x: x.year).values)
    plot_hourly_heatmap(data, countries[i], xlabels, f'Hourly imports from {countries[i]} (max power: {series.max():.0f} MW)',
                        'Consumption from source (% of the max)', val_max=100, fig=fig, ax=ax[i // 2, i % 2], relative=True)

ax[0, 0].set_title(f'Hourly total of Swiss consumption (max power: {val_max:.0f} MW)')
ax[0, 1].set_title(f'Hourly consumption of swiss electricity (max power: {data[countries[1]].max():.0f} MW)')

# Heatmap hydro
cols = ['Hydro_Pumped_Storage_CH', 'Hydro_Water_Reservoir_CH']
data2 = raw_consumption_by_src['raw_df'][cols].astype(float)
for i in range(len(cols)):
    series = data2[cols[i]]
    xlabels = list(series.resample('Y').mean().index.map(lambda x: x.year).values)
    plot_hourly_heatmap(data2, cols[i], xlabels, f'Hourly production from {cols[i]} (max power: {series.max():.0f} MW)', 'Consumption from source (% of the max)',
                        val_max=100, fig=fig, ax=ax[2, i], relative=True)

fig.tight_layout();

Heatmap of the carbon emissions associated to the Swiss consumption mix :

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 5))
# Heatmap import
data = electricity_impact['df'][['sum']].copy()
val_min = data.min().min()
val_max = data.max().max()
series = data['sum']
xlabels = list(series.resample('Y').mean().index.map(lambda x: x.year).values)
plot_hourly_heatmap(data, 'sum', xlabels,
                    'Ghg emissions of Swiss electricity consumption mix \n 2021-2022', 'GHG emissions (gCO2eq/kWh)',
                    val_min=val_min,
                    val_max=val_max, fig=fig, ax=ax, cmap='inferno_r')
fig.tight_layout();

Yearly average of carbon emissions per consumed kWh :
**Warning** the data before 2019 can be mis-estimated because of the lack of knowledge on the composition of the residual

In [ ]:
countries = ['sum', 'CH', 'DE', 'FR']
start = '2021-01-01'
end = '2021-12-31'
years_label = '2021'
data = raw_consumption_by_src['df'][countries].loc[start:end].copy()

season_names = {2: 'Spring', 3: 'Summer', 4: 'Fall', 1: 'Winter'}
data['Season'] = [date.month % 12 // 3 + 1 for date in data.index]
data['Season'] = data['Season'].map(season_names)
data['DayOfWeek'] = [0 if date.weekday() < 5 else 1 if date.weekday() == 5 else 2 for date in data.index]
grouped_data = data.groupby('Season')

season_labels = [*season_names.values()]
weekdays = ['Weekday', 'Saturday', 'Sunday']
for k in range(len(weekdays)):
    fig, ax = plt.subplots(1, len(season_labels), sharey=True, figsize=(12, 3))
    for i in range(4):
        seasonal_data = [grouped_data.get_group(season) for season in season_labels]
        seasonal_data = [s[s['DayOfWeek'] == k][data.columns[i]] for s in seasonal_data]
        plot_typical_days(seasonal_data, season_labels,
                          f'Electricity consumption on average {years_label} {weekdays[k]}',
                          'Consumption from source (MWh)',
                          fig=fig, ax=ax)
    for i in range(len(season_labels)):
        for j in range(len(ax[i].get_lines())):
            ax[i].get_lines()[j].set_color(colormap[countries[j]])
    fig.legend(['Total consumption', 'Electricity produced in CH', 'Electricity imported from DE',
                'Electricity imported from FR'], loc='upper left', ncol=4, bbox_to_anchor=(0, 0))
    plt.show()

In [ ]:
data = electricity_impact['df'][countries].loc[start:end].copy()
data['Season'] = [date.month % 12 // 3 + 1 for date in data.index]
data['Season'] = data['Season'].map(season_names)
data['DayOfWeek'] = [0 if date.weekday() < 5 else 1 if date.weekday() == 5 else 2 for date in data.index]
grouped_data = data.groupby('Season')

for k in range(len(weekdays)):
    fig, ax = plt.subplots(1, len(season_labels), sharey=True, figsize=(12, 3))
    for i in range(4):
        seasonal_data = [grouped_data.get_group(season) for season in season_labels]
        seasonal_data = [s[s['DayOfWeek'] == k][data.columns[i]] for s in seasonal_data]
        plot_typical_days(seasonal_data, season_labels,
                          f'GHG emissions of one consumed kWh on average {years_label} {weekdays[k]}',
                          'GHG emissions (gCO2eq/kWh)',
                          fig=fig, ax=ax)
    for i in range(len(season_labels)):
        for j in range(len(ax[i].get_lines())):
            ax[i].get_lines()[j].set_color(colormap[countries[j]])
    fig.legend(['Total emissions', 'From CH share', 'From DE share', 'From FR share'], loc='upper left', ncol=4,
               bbox_to_anchor=(0, 0))
    plt.show()

-- The end --